Notebook uses the Code provided by Google, in their Quickstart guide for Gemini, which is a GenAI Model.
Pupose of this Notebook is to achieve result when doing prompting with passing data of customer record.

In [1]:
!pip install -U -q google.generativeai

In [2]:
import textwrap
import numpy as np
import pandas as pd

import google.generativeai as genai
import google.ai.generativelanguage as glm

# Used to securely store your API key
#from google.colab import userdata

from IPython.display import Markdown

/Users/loginatcode/Documents/study/ws1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Get the API key, from Google Gemini


In [3]:
import os
API_KEY=os.environ['G_API_KEY']

genai.configure(api_key=API_KEY)

Let's read the File with Mutiple Customer.
Different Expenses made by them are there in the file.

In [4]:
expense_df = pd.read_parquet('./1_Million_Customer_Record.pq')
expense_df

,grocery,travel,rent,auto_insurance,hsa,medical,gas,home_furnishing,restaurant,cellphone,day,month,year,expense_date,person_id
0,440,3957,2339,18,24,45,7,17,12,8,25,5,2023,2023-05-25,10012784
1,796,4739,1806,40,19,16,5,15,14,3,1,2,2023,2023-02-01,10001251
2,323,245,1532,40,23,16,9,25,12,3,9,6,2022,2022-06-09,10042602
3,229,1428,2074,40,31,43,9,8,15,1,11,11,2022,2022-11-11,10034268
4,675,222,2696,2,47,49,9,1,4,1,23,4,2023,2023-04-23,10020619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999996,433,634,1534,1,10,28,5,8,11,4,15,6,2022,2022-06-15,10015694
999997,81,1677,2967,13,38,33,8,17,17,6,17,1,2022,2022-01-17,10034246
999998,598,2993,2341,26,45,0,6,5,11,0,12,3,2022,2022-03-12,10027764
999999,632,729,2295,44,35,49,5,19,12,5,21,5,2022,2022-05-21,10035083


In [5]:
f' Total {expense_df.person_id.nunique()} Unique Customer Record are in Customer Record file'

' Total 41979 Unique Customer Record are in Customer Record file'

In [6]:
customer_string = expense_df.loc[(expense_df.year==2023) &(expense_df.person_id==10035083),].sort_values(by='expense_date')
customer_string

,grocery,travel,rent,auto_insurance,hsa,medical,gas,home_furnishing,restaurant,cellphone,day,month,year,expense_date,person_id
446855,442,1174,2535,10,28,12,5,8,8,3,21,1,2023,2023-01-21,10035083
665850,205,2390,2801,12,18,36,7,16,7,6,2,3,2023,2023-03-02,10035083
747246,597,1412,2005,27,16,32,5,4,13,4,15,3,2023,2023-03-15,10035083
284504,597,1130,2109,31,45,48,8,23,11,6,20,3,2023,2023-03-20,10035083
444924,233,2037,2278,19,35,33,9,19,9,7,21,4,2023,2023-04-21,10035083
91414,702,4156,2423,49,26,8,7,11,9,9,26,4,2023,2023-04-26,10035083
683180,178,1593,2077,10,5,39,7,4,13,7,6,5,2023,2023-05-06,10035083
328173,141,4330,2439,37,10,47,5,7,14,9,11,5,2023,2023-05-11,10035083
667864,471,2326,2786,39,45,44,7,1,5,7,24,5,2023,2023-05-24,10035083
547939,527,3953,2171,42,34,26,8,3,9,1,26,5,2023,2023-05-26,10035083


Lets, try to build the dataframe in text format for the customer in scope.

In [7]:
customer_string_text = customer_string[['expense_date','grocery','restaurant']].reset_index()
customer_string_text['index'] = customer_string_text.index
customer_string_text = customer_string_text[['index','expense_date','grocery','restaurant']]
customer_string_text = customer_string_text.to_string(index=False)
customer_string_text

' index expense_date  grocery  restaurant\n     0   2023-01-21      442           8\n     1   2023-03-02      205           7\n     2   2023-03-15      597          13\n     3   2023-03-20      597          11\n     4   2023-04-21      233           9\n     5   2023-04-26      702           9\n     6   2023-05-06      178          13\n     7   2023-05-11      141          14\n     8   2023-05-24      471           5\n     9   2023-05-26      527           9\n    10   2023-07-15      134           8\n    11   2023-08-22      338          14\n    12   2023-11-01      673           4\n    13   2023-11-07      377          17\n    14   2023-12-27      258          14'

## Question and Answering Application

Let's try to use the text generation API to create a Q & A system. Input your own custom data below to create a simple question and answering example. You will still use the dot product as a metric of similarity.

In [8]:
class Ask_Query():

    def __init__(self,query,passage,genai_model_name='gemini-1.0-pro-latest'):
        self.query = query
        self.genai_model_name = genai_model_name
        self.passage = passage

    def make_prompt(self):
        escaped = self.passage.replace("'", "").replace('"', "")#.replace("\n", " ")
        prompt = textwrap.dedent("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
        Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
        Mutiple expense record can be there within a month and No record can also be there within a month. \
        However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
        strike a friendly and converstional tone. \
        If the passage is irrelevant to the answer, you may ignore it.
        QUESTION: '{query}'
        PASSAGE: '{passage}'

            ANSWER:
        """).format(query=self.query, passage=escaped)

        return prompt
    
    def select_genai_model(self):
        for m in genai.list_models():
            if 'generateContent' in m.supported_generation_methods:
                print(m.name)

    def generate_answer(self):
        model = genai.GenerativeModel(self.genai_model_name)
        answer = model.generate_content(Ask_Query.make_prompt(self))
        return Markdown(answer.text)

Let's start asking different question to model and see the response

In [9]:
query = 'find all relevant grocery expenses record in first quarter of 2023'
A = Ask_Query(query,customer_string_text)
A.generate_answer()

Here are all the relevant grocery expenses records in the first quarter of 2023:

* 2023-01-21, $442
* 2023-03-02, $205
* 2023-03-15, $597
* 2023-03-20, $597

In [10]:
query = 'find all relevant grocery expenses record in second quarter of 2023'
A = Ask_Query(query,customer_string_text)
A.generate_answer()

Here are all the relevant grocery expenses records in the second quarter of 2023:

- 2023-04-21	$233
- 2023-04-26	$702
- 2023-05-06	$178
- 2023-05-11	$141
- 2023-05-24	$471
- 2023-05-26	$527

Model hallucinated when answering below query

In [15]:
query = 'check all relevant expense for restaurant in forth quarter of 2023'
A = Ask_Query(query,customer_string_text)
A.generate_answer()

I'm sorry but the provided passage does not include any expense record for the fourth quarter of 2023, so I can't check the relevant expenses for the restaurant for that period.

Model this time tried to find right answer for below query

In [20]:
query = 'find all relevant expense for restaurant '
A = Ask_Query(query,customer_string_text)
A.generate_answer()

Here are all the relevant expenses for the restaurant category:

* 2023-01-21: $8
* 2023-03-02: $7
* 2023-03-15: $13
* 2023-03-20: $11
* 2023-04-21: $9
* 2023-04-26: $9
* 2023-05-06: $13
* 2023-05-11: $14
* 2023-05-24: $5
* 2023-05-26: $9
* 2023-07-15: $8
* 2023-08-22: $14
* 2023-11-01: $4
* 2023-11-07: $17
* 2023-12-27: $14

In [21]:
query = 'where is sydney located, and is there any expense for sydney location'
A = Ask_Query(query,customer_string_text)
A.generate_answer()

I apologize, but the reference text provided does not contain any information about Sydney or expenses related to Sydney, so I am unable to answer your question.